In [1]:
# Import dependencies
import requests
from pprint import pprint
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

tmdb_key = "6971e619144092b46f0572e9faf949de"

In [2]:
# Create variables to input into url for API requests
start_year = 1927
end_year = 2019
year = 0
years = [year for year in range(start_year,end_year+1)]

In [3]:
# Create lists to append API requests to, establish urls to be used, run for loop for requests & append to lists
movie_id = []
movie_title = []

base_url = f"https://api.themoviedb.org/3/discover/movie?api_key={tmdb_key}&primary_release_year="

sort_by = "&sort_by=revenue.desc"

revenue_url = f"{base_url}{year}{sort_by}"

for year in years:
    
    year_json = requests.get(f"{base_url}{year}{sort_by}").json()
    
    movie_id.append(year_json['results'][0]['id'])
    movie_title.append(year_json['results'][0]['original_title'])

In [5]:
# Create data frame based on new lists & verify results
movies_df = pd.DataFrame({
    'Year': years,
    'Movie ID': movie_id,
    'Movie Title': movie_title    
})
movies_df.head()

,Year,Movie ID,Movie Title
0,1927,287262,The Battles of the Coronel and Falkland Islands
1,1928,173689,Lights of New York
2,1929,65203,The Broadway Melody
3,1930,166307,Tom Sawyer
4,1931,3035,Frankenstein


In [6]:
# Create new data frame & add new column for revenues
revenues_df = movies_df.loc[:,['Year','Movie ID','Movie Title']]
revenues_df['Revenue'] = ""
revenues_df.head()

,Year,Movie ID,Movie Title,Revenue
0,1927,287262,The Battles of the Coronel and Falkland Islands,
1,1928,173689,Lights of New York,
2,1929,65203,The Broadway Melody,
3,1930,166307,Tom Sawyer,
4,1931,3035,Frankenstein,


In [8]:
# Iterrate through data frame to make API requests and append revenue data to data frame
base_url2 = f"https://api.themoviedb.org/3/movie/"
api_key = f"?api_key="
language = f"&language=en-US"

for index, row in revenues_df.iterrows():
    
    query_url = base_url2 + str(row['Movie ID']) + api_key + tmdb_key + language
    
    response_url = requests.get(query_url).json()
    
    try:
        revenues_df.loc[index, 'Revenue'] = response_url['revenue']
    
    except (KeyError):
        print(f"No Revenue For this film - '{row['Movie Title']}'")

In [9]:
# Verify results
revenues_df.head()

,Year,Movie ID,Movie Title,Revenue
0,1927,287262,The Battles of the Coronel and Falkland Islands,6241682
1,1928,173689,Lights of New York,1200000
2,1929,65203,The Broadway Melody,4358000
3,1930,166307,Tom Sawyer,11000000
4,1931,3035,Frankenstein,12000000


In [10]:
# Export data frame to csv
revenues_df.to_csv('Output/Top_grossing_films.csv', index = False)